## Data Preparation - Extracting restaurant menus from the website La Fourchette

In [1]:
%cd "C:\Users\Pham Antoine\Desktop\NLP_CARTES\PROJECT"

C:\Users\Pham Antoine\Desktop\NLP_CARTES\PROJECT


In [2]:
import sys
import os
import time
import numpy as np
import json
import re
import csv

In [3]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
chromedriver='C:\\Users\\Pham Antoine\\AppData\\Local\\chromedriver.exe'

options = webdriver.ChromeOptions()

In [5]:
def get_digit():
    actual_dir = os.getcwd()
    actual_dirfile=actual_dir+'\\data\\raw'
    dir_tree = os.walk(actual_dirfile)
    p=0
    endingpage=0
    endingrestaurant=0
    for dirpath, dirnames, filenames in dir_tree:
        if 'LaFourchette.txt' not in filenames:
            open(actual_dirfile+r'\\LaFourchette.txt', 'wb')
            open(actual_dirfile+r'\\LaFourchette.json', 'wb')
            p+=0
        else:
            f=open(actual_dirfile+r'\\digit.txt').read()
            f=f.split()
            endingpage += int(f[1])
            endingrestaurant += int(f[3])
            p+=1
    return p,endingpage, endingrestaurant        
    

In [6]:
def get_browser():
        options = webdriver.ChromeOptions()
        browser = webdriver.Chrome(chromedriver, options=options)
        digit=get_digit()
        lastpage=int(digit[1])
        if digit[0] == 0:
            url='https://www.lafourchette.com/'
            #url='https://www.lafourchette.com/restaurant+paris#sort=QUALITY_DESC&page=1' 
            browser.get(url)
            time.sleep(10)
            # go to  "Paris"
            browser.find_element_by_xpath("""/html/body/div[2]/main/div[1]/div/section[1]/ul/li[1]/div/div/div/h4/a""").click()
            # go to "Voir tout"
            time.sleep(5)
            browser.find_element_by_xpath("""/html/body/div[2]/main/div/div/div[2]/section[4]/a""").click()
        else:
            url='https://www.lafourchette.com/restaurant+paris#sort=QUALITY_DESC&page=%s' %str(lastpage)
            browser.get(url)
            time.sleep(10)
            WebDriverWait(browser, 10).until_not(EC.visibility_of_element_located((By.ID, "overley")))
        return browser

In [7]:
def get_OneMenu(restoUrl):
    
    try:
        textDict={}
        driver=webdriver.Chrome(chromedriver, chrome_options=options)
        driver.get(restoUrl)
        elementList=driver.find_elements_by_class_name("cardCategory")# go to content
        textList=[elementList[i].text for i in range(len(elementList))]
        textDict[restoUrl]="".join(textList)
        driver.close()
    except:
        print("no element founded")
        pass
    return textDict,"".join(textList)

In [8]:
# Put all together
    
def get_MenusAllPages(npages=None):
    '''
    npages = number of pages that will be loaded
    '''
    
    starttime = time.time()
    textbyList=[]
    textbyDict={}
    
    browser= get_browser()  
        
    def get_MenusOnePage():
        texts=[]
        text_dict={}
        i,j=1,0
        time.sleep(5)
        while True:
            try:
                restoUrl = browser.find_element_by_xpath("""//*[@id="resultsContent"]/ul/li[%d]/div[2]/h3/a""" %i).get_attribute("href")
                print("loading restaurant...%d" %i)
                print(restoUrl)
                menu = get_OneMenu(restoUrl)
                text_dict.update(menu[0])
                texts.append(menu[1])
                i+=1
                time.sleep(5)
            except NoSuchElementException:
                print("restaurant...%d does not exist" %i)
                j+=1
                i+=1
                if j<2:
                    try:
                        restoUrl = browser.find_element_by_xpath("""//*[@id="resultsContent"]/ul/li[%d]/div[2]/h3/a""" %i).get_attribute("href")
                        time.sleep(3)
                        print("loading restaurant...%d" %i)
                        print(restoUrl)
                        menu = get_OneMenu(restoUrl)
                        time.sleep(5)
                        text_dict.update(menu[0])
                        texts.append(menu[1])
                        i+=1
                        time.sleep(3)
                    except:
                        print("\nreached end of page\ntrying to reach next page...\n")
                        time.sleep(3)
                        break
                else:
                    print("\nreached end of all of pages!\n")
                    time.sleep(3)
                    break
                time.sleep(5)
                #browser.close()
        return text_dict, texts
    
    digit=get_digit()
    for _ in range(npages):
        if _ < 1:
            if digit[1] == 0:
                print("start at the 1st page")
            else:
                print('Opening page...' + str(re.findall(r'page=(.+)',browser.current_url)[0]))
            menu=get_MenusOnePage()
            textbyList.append(menu[1])
            for k,v in menu[0].items():
                textbyDict[k] = v
            with open('.\\data\\raw\\LaFourchette.txt', 'a', encoding='utf-8') as file:
                file.write("".join(menu[1]))
            with open('.\\data\\raw\\LaFourchette.json', 'a') as outfile:
                json.dump(menu[0], outfile)
        else:
            time.sleep(10)
            print(browser.current_url)
            wait = WebDriverWait(browser, 10)
            element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME,'next')))
            element.click()
            #browser.find_element_by_class_name('next').click()
            time.sleep(5)
            print('Opening page...' + str(re.findall(r'page=(.+)',browser.current_url)[0]))
            menu=get_MenusOnePage()
            textbyList.append(menu[1])
            for k,v in menu[0].items():
                textbyDict[k] = v
            with open('.\\data\\raw\\LaFourchette.txt', 'a', encoding='utf-8') as file:
                file.write("".join(menu[1]))
            with open('.\\data\\raw\\LaFourchette.json', 'a') as outfile:
                json.dump(menu[0], outfile)
            time.sleep(10)
            continue
      
    numberOfRestaurant= sum([len(textbyList[i]) for i in range(len(textbyList))])
    
    endingpage = re.findall(r'page=(.+)',browser.current_url)[0]
    
    endingrestaurant = len(textbyList[-1])
        
    numberOfLoadedPage= len(textbyList)
    
    timedelta=(time.time()-starttime)/60
    
    print("\n%d menus loaded !" %numberOfRestaurant)   
    
    print( "\nGame over at page...{} and at the {}th customer".format(endingpage,endingrestaurant))
    
    with open('.\\data\\raw\\digit.txt', 'w') as d:
        d.write("endingpage: {} endingrestaurant: {}".format(endingpage,endingrestaurant))
    
    print("\nExecution time of code: %d minutes" % timedelta)
    
    file.close()
    outfile.close()
    return textbyDict,textbyList

In [9]:
out= get_MenusAllPages(4)

start at the 1st page
loading restaurant...1
https://www.lafourchette.com/restaurant/comme-la-coree/440617


c:\program files\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


loading restaurant...2
https://www.lafourchette.com/restaurant/first-lap/434015
loading restaurant...3
https://www.lafourchette.com/restaurant/madito/395905
loading restaurant...4
https://www.lafourchette.com/restaurant/le-petit-commines/266379
loading restaurant...5
https://www.lafourchette.com/restaurant/la-table-libanaise/1670
restaurant...6 does not exist
loading restaurant...7
https://www.lafourchette.com/restaurant/resto-indiway/409459
loading restaurant...8
https://www.lafourchette.com/restaurant/chez-fernand-christine/56889
loading restaurant...9
https://www.lafourchette.com/restaurant/le-sabal/422863
loading restaurant...10
https://www.lafourchette.com/restaurant/l-abeille-shangri-la-hotel-paris/8976
loading restaurant...11
https://www.lafourchette.com/restaurant/joeric/331221
loading restaurant...12
https://www.lafourchette.com/restaurant/pratolina/301457
loading restaurant...13
https://www.lafourchette.com/restaurant/allo-tandoori/273995
loading restaurant...14
https://www.l

loading restaurant...18
https://www.lafourchette.com/restaurant/l-assiette-gourmande/277785
loading restaurant...19
https://www.lafourchette.com/restaurant/amores/74409
loading restaurant...20
https://www.lafourchette.com/restaurant/magnum/426247
loading restaurant...21
https://www.lafourchette.com/restaurant/l-apero-bio/430975
loading restaurant...22
https://www.lafourchette.com/restaurant/hao-long/215199
loading restaurant...23
https://www.lafourchette.com/restaurant/le-quinzieme/22674
loading restaurant...24
https://www.lafourchette.com/restaurant/saigon-jadis-2/426285
loading restaurant...25
https://www.lafourchette.com/restaurant/les-fines-lames/395069
loading restaurant...26
https://www.lafourchette.com/restaurant/marclee-asian-fusion-experience/48396
restaurant...27 does not exist

reached end of all of pages!


100 menus loaded !

Game over at page...4 and at the 25th customer

Execution time of code: 27 minutes


In [10]:
print(len(out[0].keys()))

100


In [11]:
out[0].keys()

dict_keys(['https://www.lafourchette.com/restaurant/comme-la-coree/440617', 'https://www.lafourchette.com/restaurant/first-lap/434015', 'https://www.lafourchette.com/restaurant/madito/395905', 'https://www.lafourchette.com/restaurant/le-petit-commines/266379', 'https://www.lafourchette.com/restaurant/la-table-libanaise/1670', 'https://www.lafourchette.com/restaurant/resto-indiway/409459', 'https://www.lafourchette.com/restaurant/chez-fernand-christine/56889', 'https://www.lafourchette.com/restaurant/le-sabal/422863', 'https://www.lafourchette.com/restaurant/l-abeille-shangri-la-hotel-paris/8976', 'https://www.lafourchette.com/restaurant/joeric/331221', 'https://www.lafourchette.com/restaurant/pratolina/301457', 'https://www.lafourchette.com/restaurant/allo-tandoori/273995', 'https://www.lafourchette.com/restaurant/la-grande-cascade-frederic-robert/122', 'https://www.lafourchette.com/restaurant/monsieur-restaurant/4447', 'https://www.lafourchette.com/restaurant/aarchna/15964', 'https://